In [1]:
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 23.5 MB/s eta 0:00:00


In [2]:
!pip install -q transformers av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 43.6 MB/s eta 0:00:00


In [3]:
!pip install -q evaluate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [4]:
pip install -q wandb

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Load WandB API key from Kaggle Secret
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")  # Ensure the key matches your Kaggle secret

In [6]:
import wandb
wandb.login(key=wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: anjilakshetri (anjilakshetri-kathmandu-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from transformers import LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor

In [8]:
from torch.utils.data import Dataset, DataLoader
import av
from PIL import Image
from torchvision import transforms

In [9]:
data = pd.read_csv('/kaggle/input/videocap/processed_data.csv')
data.head()

,path,class,questions,answers,refined_answer
0,/kaggle/input/videocap/processed_data/processe...,Bhairab Dance,What specific outfits are the people wearing?,The people are wearing traditional Bhutanese c...,The performers are adorned in traditional Newa...
1,/kaggle/input/videocap/processed_data/processe...,Bhairab Dance,Are there any cultural dances depicted?,"Yes, the frames depict a Bhairab Dance, which ...","Yes, the frames depict the Bhairab Dance, a si..."
2,/kaggle/input/videocap/processed_data/processe...,Bhairab Dance,Describe the location shown in the video.,The video appears to be set in an indoor arena...,The video appears to be set inside a tradition...
3,/kaggle/input/videocap/processed_data/processe...,Bhairab Dance,Are there many people dancing?,"Yes, there is a large number of people partici...","Yes, there is a large number of people partici..."
4,/kaggle/input/videocap/processed_data/processe...,Bhairab Dance,How does the video make you feel?,The video frames depict a vibrant and dynamic ...,The video makes you feel deeply moved by the r...


In [10]:
def reduce_data_by_class(dataframe, sample_ratio, random_state=42):
    """
    Reduces the data by sampling a specified proportion from each class.

    Args:
        dataframe (pd.DataFrame): The input DataFrame with a 'class' column.
        sample_ratio (float): The proportion of data to keep from each class.
        random_state (int): The random state to use for reproducibility.

    Returns:
        pd.DataFrame: A new DataFrame containing the sampled data.
    """
    sampled_dfs = []
    for class_name, class_df in dataframe.groupby('class'):
        num_samples = int(len(class_df) * sample_ratio)
        sampled_class_df = class_df.sample(n=num_samples, random_state=random_state)
        sampled_dfs.append(sampled_class_df)

    return pd.concat(sampled_dfs, ignore_index=True)

In [11]:
# Reduce data by class
data = reduce_data_by_class(data, sample_ratio=0.5, random_state=42) # example of keeping 50% data
print(len(data))
print(data.head())

3404
                                                path          class  \
0  /kaggle/input/videocap/processed_data/processe...  Bhairab Dance   
1  /kaggle/input/videocap/processed_data/processe...  Bhairab Dance   
2  /kaggle/input/videocap/processed_data/processe...  Bhairab Dance   
3  /kaggle/input/videocap/processed_data/processe...  Bhairab Dance   
4  /kaggle/input/videocap/processed_data/processe...  Bhairab Dance   

                                    questions  \
0   How many people are dancing in the video?   
1  Where are the people located in the video?   
2        What is the count of people dancing?   
3         What kind of atmosphere is present?   
4       What dance movements can be observed?   

                                             answers  \
0  In the video, there are multiple individuals p...   
1  The people are located in an outdoor setting, ...   
2  The count of people dancing is not clearly dis...   
3  The atmosphere present is vibrant and festive,

In [12]:
from transformers import BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load the model in 4-bit precision
    bnb_4bit_quant_type="nf4",  # Use 4-bit NormalFloat quantization
    bnb_4bit_compute_dtype=torch.float16,  # Compute in float16 for efficiency
    bnb_4bit_use_double_quant=True,  # Use double quantization for better accuracy
)

In [13]:
processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf", quantization_config=quantization_config, device_map="auto")

preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [14]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [15]:
# TODO check this
if hasattr(model, "enable_input_require_grads"):
  model.enable_input_require_grads()
else:
    def make_inputs_require_grad(module, input, output):
        output.requires_grad_(True)
    model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)

In [16]:
from peft import LoraConfig, get_peft_model

def find_all_linear_names(model):
    cls = torch.nn.Linear
    lora_module_names = set()

    # TODO check this
    multimodal_keywords = ['mm_projector', 'vision_tower', 'vision_resampler']
    for name, module in model.named_modules():
        if any(mm_keyword in name for mm_keyword in multimodal_keywords):
            continue
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=find_all_linear_names(model),
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

In [17]:
from tqdm import tqdm  # Import tqdm for progress tracking

class VideoCaptioningDataset(Dataset):
    def __init__(self, dataframe, processor, num_frames=2):
        self.dataframe = dataframe
        self.processor = processor
        self.num_frames = num_frames
        self.resize_transform = transforms.Resize((224, 224), antialias=True) # Initialize resize transform


    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
    
        # Load video and sample frames
        video_path = row['path']
        container = av.open(video_path)
        total_frames = container.streams.video[0].frames
        indices = np.linspace(0, total_frames - 1, num=self.num_frames, dtype=int)
        video_frames = self.read_video_pyav(container, indices)
    
        # Prepare the conversation
        question = row['questions']
        refined_answer = row['refined_answer']
    
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {"type": "video"},
                ],
            },
            {
                "role": "assistant",
                "content": refined_answer,
            },
        ]
    
        # Format the conversation into a string
        formatted_text = self.format_conversation(conversation)
    
        # Tokenize the formatted text and process the video
        inputs = self.processor(
            text=formatted_text,  # Pass the formatted string
            videos=video_frames,  # Pass the video frames
            return_tensors="pt",  # Return PyTorch tensors
            padding=True,  # Pad sequences to the same length
            truncation=True,  # Truncate sequences that are too long
        )
    
        # Add labels for supervised training
        inputs["labels"] = inputs["input_ids"].clone()  # Use input_ids as labels for language modeling
    
        # Flatten the inputs for the model
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}
        return inputs

    def read_video_pyav(self, container, indices):
        """
        Decode the video with PyAV decoder.
        Args:
            container (`av.container.input.InputContainer`): PyAV container.
            indices (`List[int]`): List of frame indices to decode.
        Returns:
            result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
        """
        frames = []
        container.seek(0)
        start_index = indices[0]
        end_index = indices[-1]
        for i, frame in enumerate(container.decode(video=0)):
            if i > end_index:
                break
            if i >= start_index and i in indices:
                frame = frame.to_ndarray(format="rgb24")
                frame = Image.fromarray(frame)
                frame = self.resize_transform(frame)
                frames.append(np.array(frame))
        return np.stack(frames)


    def format_conversation(self, conversation):
        """
        Convert the conversation list of dictionaries into a single string.
        """
        formatted_text = ""
        for turn in conversation:
            role = turn["role"]
            content = turn["content"]
            if role == "user":
                for item in content:
                    if item["type"] == "text":
                        formatted_text += f"User: {item['text']}\n"
                    elif item["type"] == "video":
                        formatted_text += "<video>\n"
            elif role == "assistant":
                formatted_text += f"Assistant: {content}\n"
        return formatted_text

In [18]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    """
    Custom collate function to pad sequences in a batch.
    """
    # Separate inputs into individual components
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    videos = [item['pixel_values_videos'] for item in batch]  # Use the correct key
    labels = [item['labels'] for item in batch]


    # Pad input_ids and attention_mask
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=processor.tokenizer.pad_token_id)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100) # Padding value of -100 because we will be ignoring it later

    # Stack videos (assumes all videos have the same shape)
    videos_stacked = torch.stack(videos)

    # Return the padded batch
    return {
        'input_ids': input_ids_padded,
        'attention_mask': attention_mask_padded,
        'pixel_values_videos': videos_stacked,  # Use the correct key
        'labels': labels_padded,
    }

In [19]:
# # Create the dataset
# dataset = VideoCaptioningDataset(dataframe=data, processor=processor, num_frames=2)

# # Create the DataLoader
# dataloader = DataLoader(
#     dataset,
#     batch_size=1,
#     shuffle=True,
#     collate_fn=custom_collate_fn,  # Use the custom collate function
# )

In [20]:
from transformers import Trainer, TrainingArguments

In [21]:
# # Iterate through the DataLoader with TQDM
# debug_batches = 3  # Number of batches to debug
# if i < debug_batches:
#     for i, batch in enumerate(tqdm(dataloader, desc="Processing batches")):
#         # Inspect the batch (only for the first few batches)
#         print("Batch keys:", batch.keys())
#         print("Input IDs shape:", batch['input_ids'].shape)
#         print("Attention mask shape:", batch['attention_mask'].shape)
#         print("Video frames shape:", batch['pixel_values_videos'].shape)

In [22]:
from sklearn.model_selection import train_test_split

#Shuffle the dataset
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

#Split the dataset
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Create training and validation datasets
train_dataset = VideoCaptioningDataset(dataframe=train_data, processor=processor, num_frames=2)
val_dataset = VideoCaptioningDataset(dataframe=val_data, processor=processor, num_frames=2)

In [23]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader
# Create the validation DataLoader with custom collate
val_dataloader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,  #No need to shuffle for validation
    collate_fn=custom_collate_fn,
)

In [24]:
len(train_dataset)

2723

In [25]:
from tqdm.auto import tqdm  # Import tqdm for progress tracking

# Define training arguments
training_args = TrainingArguments(
    run_name="llava-next-video-finetune",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_steps = 5,
    learning_rate = 1e-5,
    fp16=False,
    bf16=False,
    max_steps=120,
    
    logging_steps=10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    save_steps=20,

    output_dir="./results",
    logging_dir="./logs",
        
    gradient_checkpointing=True,  # Disable gradient checkpointing if causing issues
    report_to='wandb',
)

In [26]:
from transformers import TrainerCallback

class TQDMProgressBar(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        self.pbar = tqdm(total=state.max_steps, desc="Training Progress")
    
    def on_step_end(self, args, state, control, **kwargs):
        self.pbar.update(1)
    
    def on_train_end(self, args, state, control, **kwargs):
        self.pbar.close()

In [27]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Training dataset
    data_collator=custom_collate_fn,
)

In [28]:
trainer.train()

wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250125_205613-am4c45br
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run llava-next-video-finetune
wandb: ⭐️ View project at https://wandb.ai/anjilakshetri-kathmandu-university/huggingface
wandb: 🚀 View run at https://wandb.ai/anjilakshetri-kathmandu-university/huggingface/runs/am4c45br
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,12.107100
20,11.404700
30,10.517900
40,9.347300
50,8.271600
60,7.263400
70,6.529100
80,5.840000
90,5.327600
100,5.010500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=120, training_loss=7.605635865529378, metrics={'train_runtime': 7855.4891, 'train_samples_per_second': 0.122, 'train_steps_per_second': 0.015, 'total_flos': 1.5089386416427008e+16, 'train_loss': 7.605635865529378, 'epoch': 0.35255233198677927})

In [29]:
# Save the fine-tuned model and processor in Kaggle
output_dir = "/kaggle/working/fine-tuned-llava-next-video"
trainer.save_model(output_dir)
processor.save_pretrained(output_dir)
processor.tokenizer.save_pretrained(output_dir)

print(f"Model and processor saved to {output_dir}")

Model and processor saved to /kaggle/working/fine-tuned-llava-next-video


In [30]:
# # Evaluate the model on the validation dataset
# eval_results = trainer.evaluate()

# # Print the evaluation results
# print("Evaluation results:", eval_results)